![ooops](https://billswire.usatoday.com/wp-content/uploads/sites/65/2021/01/USATSI_15479143.jpg?w=1000&h=600&crop=1)

We want to estimate ability to catch the ball (for opponent receiver)

# I. Collect the data

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# read income data
game = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/games.csv')
player = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/players.csv')
play = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/plays.csv')

tracking2018 = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/tracking2018.csv')
tracking2019 = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/tracking2019.csv')
tracking2020 = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/tracking2020.csv')

tracking = tracking2018
tracking.append(tracking2019)
tracking.append(tracking2020)

PFF = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/PFFScoutingData.csv')

# II. Determine player
Who was the nearest to ball

In [ ]:
import math
punt = pd.merge(play[play.specialTeamsPlayType=='Punt'],
                 game,
                 how = "inner", on = 'gameId')
kickoff = pd.merge(play[play.specialTeamsPlayType=='Kickoff'],
                 game,
                 how = "inner", on = 'gameId')

# function to determine nearest receiver
def nearest_to_ball(_x,_y,_team,_gameId,_playId):
    global tracking
    subplay = tracking[(tracking.playId ==_playId) & (tracking.gameId ==_gameId)
                       &(tracking.team==_team)]
    subplay = subplay.reset_index()
    min_dist=170**2
    playerId=0
    for i in range(len(subplay)):
        if ((subplay['x'][i]-_x)**2+(subplay['y'][i]-_y)**2)<min_dist:
            min_dist= ((subplay['x'][i]-_x)**2+(subplay['y'][i]-_y)**2)
            playerId= subplay['nflId'][i]
    return playerId

In [ ]:
#punt muffed
punt_muffed = punt[punt.specialTeamsResult == 'Muffed']
punt_muffed = pd.merge(punt_muffed[['gameId', 'playId', 'possessionTeam','homeTeamAbbr','visitorTeamAbbr']],
                  tracking[(tracking.team == 'football') & (tracking.event=='punt_muffed')],
                  how = 'inner', on = ['gameId','playId']                 
                 )
punt_muffed = punt_muffed[['gameId', 'playId', 'possessionTeam','homeTeamAbbr','visitorTeamAbbr','x','y']]

#kickoff muffed
kickoff_muffed = kickoff[kickoff.specialTeamsResult == 'Muffed']
kickoff_muffed = pd.merge(kickoff_muffed[['gameId', 'playId', 'possessionTeam','homeTeamAbbr','visitorTeamAbbr']],
                  tracking[(tracking.team == 'football') & (tracking.event=='punt_muffed')],
                  how = 'inner', on = ['gameId','playId']                 
                 )
kickoff_muffed = kickoff_muffed[['gameId', 'playId', 'possessionTeam','homeTeamAbbr','visitorTeamAbbr','x','y']]

Blunders on punt

In [ ]:
nearest_nflid = []
for i in range(len(punt_muffed)):
    if punt_muffed['possessionTeam'][i]==punt_muffed['visitorTeamAbbr'][i]:
        nearest_nflid.append(nearest_to_ball(punt_muffed['x'][i],punt_muffed['y'][i],
                                             "home",punt_muffed['gameId'][i],punt_muffed['playId'][i]))
    elif punt_muffed['possessionTeam'][i]==punt_muffed['homeTeamAbbr'][i]:
        nearest_nflid.append(nearest_to_ball(punt_muffed['x'][i],punt_muffed['y'][i],
                                             "away",punt_muffed['gameId'][i],punt_muffed['playId'][i]))
    else:
        pass
p_cnt = np.ones(len(punt_muffed))
k_cnt = np.zeros(len(punt_muffed))
punt_muffed = pd.concat([punt_muffed,
                    pd.DataFrame(nearest_nflid, columns = ['nflId']),
                    pd.DataFrame(p_cnt,columns=['punt_cnt']),
                    pd.DataFrame(k_cnt, columns=['kickoff_cnt'])
                   ],
                  axis=1)
punt_muffed.head(5)


Blunders on kickoff

In [ ]:
nearest_nflid = []
for i in range(len(kickoff_muffed)):
    if kickoff_muffed['possessionTeam'][i]==kickoff_muffed['visitorTeamAbbr'][i]:
        nearest_nflid.append(nearest_to_ball(kickoff_muffed['x'][i],kickoff_muffed['y'][i],
                                             "home",kickoff_muffed['gameId'][i],kickoff_muffed['playId'][i]))
    elif kickoff_muffed['possessionTeam'][i]==kickoff_muffed['homeTeamAbbr'][i]:
        nearest_nflid.append(nearest_to_ball(kickoff_muffed['x'][i],kickoff_muffed['y'][i],
                                             "away",kickoff_muffed['gameId'][i],kickoff_muffed['playId'][i]))
    else:
        pass
p_cnt = np.zeros(len(kickoff_muffed))
k_cnt = np.ones(len(kickoff_muffed))
kickoff_muffed = pd.concat([kickoff_muffed,
                    pd.DataFrame(nearest_nflid, columns = ['nflId']),
                    pd.DataFrame(p_cnt,columns=['punt_cnt']),
                    pd.DataFrame(k_cnt, columns=['kickoff_cnt'])
                   ],
                  axis=1)
kickoff_muffed.head(5)

In [ ]:
muffed = punt_muffed
muffed.append(kickoff_muffed)

muffed['total'] = muffed['punt_cnt']+muffed['kickoff_cnt']

# III. Results

In [ ]:
pp = pd.merge(muffed, player, how='inner', on ='nflId')
pp= pp[['nflId','punt_cnt','kickoff_cnt','total','displayName','Position']]

import seaborn as sns
ax = sns.displot(data=pp[['Position']],x="Position", binwidth=5, 
                 #hue="specialTeamsResult", 
                 multiple="stack")
ax.set(title='Muffed by position')

In [ ]:
pp[['displayName','total']].groupby(['displayName']).sum('total').sort_values(by = 'total', ascending=False)

May be there is a weak link.